# The Five-Line Recommender, Explained

Building a recommender system is easy with GraphLab Create.  Simply import graphlab, load data, create a recommender model, and start making recommendations.  Let's walk through this line by line.

<h3>Step 1: Import GraphLab into Python</h3>

In [1]:
import graphlab

<h3>Step 2: Load the dataset</h3>

The data is sitting on an AWS S3 bucket as a csv file.  We can load it into a GraphLab SFrame with `read_csv()`, specifying the "rating" column to be loaded as integers.  For other ways of creating an SFrame and doing data munging, see the <a href="https://dato.com/learn/notebooks/introduction_to_sframes.html">SFrame tutorial</a>.

In [2]:
data = graphlab.SFrame.read_csv("http://s3.amazonaws.com/dato-datasets/movie_ratings/training_data.csv", column_type_hints={"rating":int})
data.head()

[INFO] This non-commercial license of GraphLab Create is assigned to drbaguiar@gmail.comand will expire on October 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-7996 - Server binary: C:\Users\bryan_000\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\BRYAN_~1\AppData\Local\Temp\graphlab_server_1445824338.log.0
[INFO] GraphLab Server Version: 1.6.1


PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/movie_ratings/training_data.csv to C:/Users/BRYAN_~1/AppData/Local/Temp/graphlab-bryan_000/7996/1536ec3d-eb41-41fc-abaf-e7e20a7a449b.csv
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/movie_ratings/training_data.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.161094 secs.
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/movie_ratings/training_data.csv
PROGRESS: Parsing completed. Parsed 82068 lines in 0.128092 secs.


user,movie,rating
Jacob Smith,Flirting with Disaster,4
Jacob Smith,Indecent Proposal,3
Jacob Smith,Runaway Bride,2
Jacob Smith,Swiss Family Robinson,1
Jacob Smith,The Mexican,2
Jacob Smith,Maid in Manhattan,4
Jacob Smith,A Charlie BrownThanksgiving / The ...,3
Jacob Smith,Brazil,1
Jacob Smith,Forrest Gump,3
Jacob Smith,It Happened One Night,4


<h3>Step 3: Build a model</h3>

We have the data.  It's time to build a model.  Now, have you ever tried typing "item similarity, matrix factorization, factorization machine, popularity" ten times in a row?  We have, and we don't recommend it.  (Harhar.)  There are many good models for making recommendations, but sometimes even knowing the right names can be a challenge, much less typing them time after time.

This is why GraphLab Create provides a default recommender called ... `recommender`.  You can build a default recommender with `recommender.create()`.  It requires a dataset to use for training the model, as well as the names of the columns that contain the user IDs, the item IDs, and the ratings (if present).

In [3]:
# Build a default recommender (a Matrix Factorization model)
# The data needs to contain at least three columns: user, item, and rating.
# All other columns in the dataset are ignored by the default recommender.
model = graphlab.recommender.create(data, user_id="user", item_id="movie", target="rating")

PROGRESS: Recsys training: model = ranking_factorization_recommender
PROGRESS: Preparing data set.
PROGRESS:     Data has 82068 observations with 334 users and 7714 items.
PROGRESS:     Data prepared in: 0.206147s
PROGRESS: Training ranking_factorization_recommender for recommendations.
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | Parameter                      | Description                                      | Value    |
PROGRESS: +--------------------------------+--------------------------------------------------+----------+
PROGRESS: | num_factors                    | Factor Dimension                                 | 32       |
PROGRESS: | regularization                 | L2 Regularization on Factors                     | 1e-009   |
PROGRESS: | solver                         | Solver used for training                         | sgd      |
PROGRESS: | linear_regularization          | L2 Regularization on Line

<b>Details</b> (and the small devil therein):

Under the hood, the type of recommender is chosen based on the provided data and whether the desired task is ranking (default) or rating prediction.  The default recommender for this type of data and the default ranking task is a matrix factorization model, implemented on top of the disk-backed SFrame data structure.  The default solver is stochastic gradient descent, and the recommender model used is the <a href="https://dato.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_model.RankingFactorizationModel.html"> RankingFactorizationModel </a>, which balances rating prediction with a ranking objective.  The default `create()` function does not allow changes to the default parameters of a specific model, but it is just as easy to build a specific recommender with your own parameters using the appropriate model-specific `create()` function. 

<h3>Step 4: Make recommendations</h3>

The trained model can now make recommendations of new items for users.  To do so, call `recommend()` with an SArray of user ids.  If `users` is set to None, then `recommend()` will make recommendations for all the users seen during training, automatically excluding the items that are observed for each user.  In other words, if `data` contains a row "Alice, The Great Gatsby", then `recommend()` will not recommend "The Great Gatsby" for user "Alice".  It will return at most `k` new items for each user, sorted by their rank.  It will return fewer than `k` items if there are not enough items that the user has not already rated or seen.

The `score` column of the output contains the *unnormalized* prediction scores for each user-item pair.  The semantic meanings of these scores may differ between models.  For the linear regression model, for instance, a higher average score for a user means that the model thinks that this user is generally more enthusiastic than others.  See the <a href="https://dato.com/learn/notebooks/basic_recommender_functionalities.html">Basic Recommender Functionalities tutorial</a> for more details.

In [4]:
# You can now make recommendations for all the users you've just trained on
results = model.recommend(users=None, k=5)
results.head()

user,movie,score,rank
Jacob Smith,To Kill a Mockingbird,4.73138736284,1
Jacob Smith,Mr. Smith Goes toWashington ...,4.4389288286,2
Jacob Smith,One Flew Over theCuckoo's Nest ...,4.36014317072,3
Jacob Smith,West Side Story,4.33668516672,4
Jacob Smith,Sabrina,4.24056695497,5
Mason Smith,Dog Day Afternoon,5.74551676309,1
Mason Smith,Fahrenheit 9/11,5.5973539213,2
Mason Smith,Rushmore,5.49868963754,3
Mason Smith,Napoleon Dynamite,5.42212055719,4
Mason Smith,The Sound of Music,5.40702484644,5


<h3>Step 5: Save the model</h3>

The model can be saved for later use, either on the local machine or in an AWS S3 bucket.  The saved model sits in its own directory, and can be loaded back in later to make more predictions.


In [5]:
# Save the model for later use
model.save("my_model")

Et voil&agrave;! You've just built your first recommender with GraphLab Create.  Congratulations!

(Looking for more details about the modules and functions? Check out the <a href="https://dato.com/products/create/docs/">API docs</a>.)